In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [13]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train=list(X_train)
sess=tf.Session()
for i in range(10):
    x_batch=tf.train.batch(X_train,batch_size=5).eval(sess)
    print(x_batch)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


AttributeError: 'list' object has no attribute 'eval'

In [2]:
#cifar10 실습
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
train_x_list=[]
train_y_list=[]
for i in range(1,6):
    a=unpickle("cifar-10-batches-py/data_batch_"+str(i))
    train_x_list.append(np.array(a[b'data']))
    train_y_list.append(np.array(a[b'labels']).reshape(-1,1))
test=unpickle("cifar-10-batches-py/test_batch")
test_x=np.array(test[b'data'])
#test_x=np.array(train_x_list).flatten('C').reshape(-1,3072).shape
test_y=np.array(test[b'labels']).reshape(-1,1)

keep_prob=tf.placeholder(tf.float32)

X=tf.placeholder(tf.float32,[None,3072])
X_img=tf.reshape(X,[-1,32,32,3])
Y=tf.placeholder(tf.int32,[None,1])
Y_one_hot=tf.one_hot(Y,10)
Y_one_hot=tf.reshape(Y_one_hot,[-1,10])

W1=tf.Variable(tf.random_normal([3,3,3,32],stddev=0.01))
L1=tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
L1=tf.nn.relu(L1)
L1=tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L1=tf.nn.dropout(L1,keep_prob=keep_prob)

W2=tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2=tf.nn.relu(L2)
L2=tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L2=tf.nn.dropout(L2,keep_prob=keep_prob)

W3=tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01))
L3=tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
L3=tf.nn.relu(L3)
L3=tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
L3=tf.nn.dropout(L3,keep_prob=keep_prob)
L3=tf.reshape(L3,[-1,4*4*128])

W4=tf.get_variable("W4",shape=[4*4*128,10],initializer=tf.contrib.layers.xavier_initializer())
B=tf.Variable(tf.random_normal([10]))
L4=tf.matmul(L3,W4)+B
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=L4,labels=Y_one_hot))
optimizer=tf.train.AdamOptimizer(0.01).minimize(cost)

is_correct=tf.equal(tf.argmax(L4,1),tf.argmax(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(15):
        avg_cost=0
        for m,n in zip(train_x_list,train_y_list):
            c,_=sess.run([cost,optimizer],feed_dict={X:m,Y:n,keep_prob:0.7})
            avg_cost+=c/5
        print("cost: ",avg_cost)
    a=sess.run(accuracy,feed_dict={X:test_x,Y:test_y,keep_prob:1})
    print('accuracy=',a)

cost:  28.507823848724367
cost:  2.790049457550049
cost:  2.7609289646148683
cost:  2.7295395851135256
cost:  2.6981127738952635
cost:  2.667583179473877
cost:  2.6384212017059325
cost:  2.610879516601563
cost:  2.5850663661956785
cost:  2.561029624938965
cost:  2.5387156009674072
cost:  2.518111753463745
cost:  2.499084663391113
cost:  2.481599760055542
cost:  2.465517663955689
accuracy= 0.1
